In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=9d583cd29c618139d6824bd4cad2f4b881ed7c41d3e565ce1240c53a8bb2944e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# let's now print how the Dataset looks like
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})

In [4]:
raw_datasets.column_names

{'train': ['tokens', 'ner_tags'],
 'test': ['tokens', 'ner_tags'],
 'validation': ['tokens', 'ner_tags']}

In [5]:
# let's print an instance of dataset
idx=1000
rec=raw_datasets['train'][idx]
for w, t in zip(rec['tokens'],rec['ner_tags']):
  print('{}\t{}'.format(w,t))

सूक्ष्म	0
,	0
लघु	0
एवं	0
मध्यम	0
उद्यम	0
(	0
एमएसएमई	3
)	0
और	0
सड़क	3
परिवहन	4
एवं	4
राजमार्ग	4
मंत्री	0
श्री	0
नितिन	1
गडकरी	2
ने	0
आज	0
जानकारी	0
दी	0
कि	0
उनका	0
मंत्रालय	0
एक	0
कृषि	0
एमएसएमई	0
नीति	0
लाने	0
पर	0
काम	0
कर	0
रहा	0
है	0
जो	0
स्थानीय	0
कच्चे	0
माल	0
का	0
उपयोग	0
करते	0
हुए	0
विनिर्माण	0
उत्पादों	0
के	0
लिए	0
ग्रामीण	0
,	0
जनजातीय	0
,	0
कृषि	0
और	0
वन	0
क्षेत्रों	0
में	0
उद्यमिता	0
विकास	0
पर	0
ध्यान	0
केंद्रित	0
करेगी	0
।	0


In [6]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [7]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [8]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [9]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

2024-03-13 08:59:18.854220: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 08:59:18.854361: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 08:59:18.980161: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

model=model.to("cuda")

In [11]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [12]:
ot=len(raw_datasets["train"])
nts=int(ot*0.2)
train_dataset = raw_datasets["train"].select(range(20000))
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/5 [00:00<?, ?ba/s]

In [13]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [14]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [15]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [16]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)

args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    learning_rate=2e-5
)

In [17]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

In [18]:
trainer.args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_

In [19]:
train_result = trainer.train()
metrics = train_result.metrics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.307300,0.307635,0.669025,0.657300,0.663111,10213,0.571916,0.427856,0.489507,9786,0.641357,0.644209,0.642780,10568,0.633107,0.579318,0.605019,0.907753
2,0.254500,0.277563,0.712666,0.683149,0.697595,10213,0.529928,0.542816,0.536295,9786,0.715007,0.656416,0.684460,10568,0.651376,0.628979,0.639981,0.914277
3,0.209500,0.274750,0.706191,0.711446,0.708809,10213,0.556893,0.539138,0.547871,9786,0.701604,0.687263,0.694359,10568,0.657646,0.647921,0.652747,0.916768


In [20]:
model.save_pretrained('my_indicBERT')

In [21]:
tokenizer.save_pretrained("tokenizer_indicBERT")

('tokenizer_indicBERT/tokenizer_config.json',
 'tokenizer_indicBERT/special_tokens_map.json',
 'tokenizer_indicBERT/spiece.model',
 'tokenizer_indicBERT/added_tokens.json',
 'tokenizer_indicBERT/tokenizer.json')

In [22]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [23]:
import json

In [24]:
config = json.load(open("/kaggle/working/my_indicBERT/config.json"))

In [25]:
config["id2label"] = id2label
config["label2id"] = label2id

In [26]:
json.dump(config, open("/kaggle/working/my_indicBERT/config.json","w"))

In [27]:
model = AutoModelForTokenClassification.from_pretrained('/kaggle/working/my_indicBERT')

In [28]:
model

AlbertForTokenClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, b

In [29]:
!pip install torch
import torch

In [30]:
def get_ner(sentence):
    tok_sentence = tokenizer(sentence, return_tensors='pt')

    with torch.no_grad():
        logits = model(**tok_sentence).logits.argmax(-1)
        predicted_tokens_classes = [
            model.config.id2label[t.item()] for t in logits[0]]

        predicted_labels = []

        previous_token_id = 0
        word_ids = tok_sentence.word_ids()
        for word_index in range(len(word_ids)):
            if word_ids[word_index] == None:
                previous_token_id = word_ids[word_index]
            elif word_ids[word_index] == previous_token_id:
                previous_token_id = word_ids[word_index]
            else:
                predicted_labels.append(predicted_tokens_classes[word_index])
                previous_token_id = word_ids[word_index]

        ner_output = []
        for index in range(len(sentence.split(' '))):
            if(index<len(predicted_labels)):
                ner_output.append((sentence.split(' ')[index], predicted_labels[index]))
            else:
                ner_output.append((sentence.split(' ')[index], 'O'))
        return ner_output

In [31]:
sentence="लगातार किसान इस बाबत जेडीए में आकर योजना को पूर्ण करने के लिये मांग कर रहे है।"
output=get_ner(sentence)
print(output)

[('लगातार', 'O'), ('किसान', 'O'), ('इस', 'O'), ('बाबत', 'O'), ('जेडीए', 'O'), ('में', 'O'), ('आकर', 'O'), ('योजना', 'O'), ('को', 'O'), ('पूर्ण', 'O'), ('करने', 'O'), ('के', 'O'), ('लिये', 'O'), ('मांग', 'O'), ('कर', 'O'), ('रहे', 'O'), ('है।', 'O')]


In [32]:
!ls

my_indicBERT  output_dir  tokenizer_indicBERT  wandb


In [33]:
!zip -r BertTuned.zip /kaggle/working/my_indicBERT

  adding: kaggle/working/my_indicBERT/ (stored 0%)
  adding: kaggle/working/my_indicBERT/config.json (deflated 51%)
  adding: kaggle/working/my_indicBERT/model.safetensors (deflated 7%)


In [34]:
!zip -r BertTuned_tokenizer.zip /kaggle/working/tokenizer_indicBERT

  adding: kaggle/working/tokenizer_indicBERT/ (stored 0%)
  adding: kaggle/working/tokenizer_indicBERT/tokenizer.json (deflated 77%)
  adding: kaggle/working/tokenizer_indicBERT/spiece.model (deflated 60%)
  adding: kaggle/working/tokenizer_indicBERT/tokenizer_config.json (deflated 74%)
  adding: kaggle/working/tokenizer_indicBERT/special_tokens_map.json (deflated 49%)


In [35]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

  0%|          | 0/3 [00:00<?, ?it/s]

In [36]:
test_dataset = raw_datasets["test"]
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [37]:
pip install --upgrade transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 69.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1
Note: you may need to restart the kernel to use updated packages.


In [38]:
predictions, labels, metrics = trainer.predict(tokenized_test_dataset)

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall']=metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1']=metrics[key]

In [39]:
print(metrics)

{'test_loss': 0.23024629056453705, 'test_LOC_precision': 0.6887755102040817, 'test_LOC_recall': 0.6596091205211726, 'test_LOC_f1': 0.6738768718801997, 'test_LOC_number': 614, 'test_ORG_precision': 0.6018018018018018, 'test_ORG_recall': 0.6361904761904762, 'test_ORG_f1': 0.6185185185185186, 'test_ORG_number': 525, 'test_PER_precision': 0.7419768934531451, 'test_PER_recall': 0.7316455696202532, 'test_PER_f1': 0.7367750159337159, 'test_PER_number': 790, 'test_overall_precision': 0.6852237252861603, 'test_overall_recall': 0.6827371695178849, 'test_overall_f1': 0.6839781874837704, 'test_overall_accuracy': 0.9293721409541045, 'test_runtime': 17.8761, 'test_samples_per_second': 48.501, 'test_steps_per_second': 6.098}


In [40]:
macro_f1 = metrics['test_LOC_f1'] + metrics['test_ORG_f1'] + metrics['test_PER_f1']
print("macro f1 is", macro_f1/3)

macro f1 is 0.6763901354441447


In [41]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokenizer_indicBERT")

model = AutoModelForTokenClassification.from_pretrained("/kaggle/working/my_indicBERT")

In [43]:
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file.readlines()]

ground_truth_file = "/kaggle/input/govind-1/govind_1.txt"
corpus = read_file(ground_truth_file)

def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)
    
    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]
    
    predicted_labels = []
    
    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]
    
    return predicted_labels

for sentence in corpus:
    temp = get_ner(sentence)
    print(temp)

[('20/08/2014', 'O'), ('मुख्य', 'O'), ('अभियंता,', 'O'), ('धसान', 'O'), ('केन', 'O'), ('कछार', 'O'), ('सागर', 'O'), ('के', 'O'), ('अन्तर्गत', 'O'), ('गुण', 'O'), ('नियंत्रण', 'O'), ('के', 'O'), ('कार्यों', 'O'), ('को', 'O'), ('गति', 'O'), ('देने', 'O'), ('हेतु', 'O'), ('मुख्यालय', 'O'), ('परिवर्तन।', 'O')]
[('जानकारी', 'O'), ('है', 'O'), ('कि', 'O'), ('प्रशांत', 'B-PER'), ('किशोर', 'I-PER'), ('ने', 'O'), ('इस', 'O'), ('काम', 'O'), ('के', 'O'), ('लिए', 'O'), ('राहुल', 'B-PER'), ('गांधी', 'I-PER'), ('से', 'O'), ('कोई', 'O'), ('भी', 'O'), ('वेतन', 'O'), ('नहीं', 'O'), ('ले', 'O'), ('रहे', 'O'), ('हैं.', 'O')]
[('यह', 'O'), ('सभी', 'O'), ('वनवासी', 'O'), ('बंधु', 'O'), ('हमारे', 'O'), ('लिए', 'O'), ('नारायण', 'O'), ('का', 'O'), ('ही', 'O'), ('रूप', 'O'), ('हैं।', 'O')]
[('उन्होंने', 'O'), ('मजदूरों', 'O'), ('के', 'O'), ('लिए', 'O'), ('जो', 'O'), ('संबल', 'O'), ('योजना', 'O'), ('बनी', 'O'), ('थी,', 'O'), ('वो', 'O'), ('बंद', 'O'), ('कर', 'O'), ('दी।', 'O')]
[('वहीं', 'O'), ('प्रबंधन', 'O'),